In [483]:
import pandas as pd
import numpy as np
import matplotlib as matplb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns

In [484]:
titanic_df = pd.read_csv('titanic/train.csv')
X_test = pd.read_csv('titanic/test.csv')
titanic_df_test = pd.read_csv('titanic/gender_submission.csv')
y_test = titanic_df_test['Survived']
y = titanic_df[['Survived']]
X = titanic_df.drop(columns = ['Survived'])

In [485]:
def pre_processs(X) :
    X = X.drop(columns = ['Ticket','Name','Cabin','PassengerId'])
    mapping = {'female': 0, 'male': 1}
    X = X.replace({'Sex': mapping})
    mapping = {'C': 0, 'S': 1, 'Q': 2}
    X = X.replace({'Embarked': mapping})
    X["Age"].fillna(X.groupby(['Pclass','Sex'])['Age'].transform("mean"), inplace=True)
    X['Age'] = pd.cut(X.Age,bins=[0,4,17,38,99],labels=[1,2,3,4])

    X['Fare'] = pd.cut(X.Fare,bins=[-1,20,40,60,80,100,1000],labels=[1,2,3,4,5,6])
    return X

In [486]:
X = pre_processs(X)
X['Embarked'].loc[61] = 1.0
X['Embarked'].loc[829] = 1.0

/Users/aditya/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [487]:
pId = X_test['PassengerId']

In [488]:
X_test = pre_processs(X_test)

In [489]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.2,max_depth=1, random_state=3).fit(X, y['Survived'])

In [490]:
y_pred = model.predict(X_test)

In [491]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.06%


In [492]:
titanic_df_pred = pd.DataFrame(list(zip(pId, y_pred)),columns =['PassengerId', 'Survived'])

In [493]:
titanic_df_pred.to_csv("titanic_df_pred.csv", index=False)